In [1]:
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

In [22]:
#Read in the data
dfBase = pd.read_csv("Data/OffensiveStats2017.csv")
dfBase = dfBase.dropna()

#Get rid of players with AB < 50
dfBase = dfBase.drop(dfBase[dfBase['AB'] < 500].index)

#Drop some stuff that was type object from original sheet
dfBase = dfBase.drop('SO%', 1)
dfBase = dfBase.drop('BB%', 1)
dfBase = dfBase.drop('SB%', 1)

dfBase['SOper'] = dfBase['SO']/dfBase['AB']
dfBase['BBper'] = dfBase['BB']/dfBase['AB']
dfBase['HRper'] = dfBase['HR']/dfBase['AB']
dfBase['SBper'] = dfBase['SB']/(dfBase['SB']+dfBase['CS'])
dfBase['RBIperAB'] = dfBase['RBI']/dfBase['AB']

dfBase['RunsGenerated'] = (.35*(dfBase['TB']+dfBase['BB'])) - (.25*(dfBase['AB']-dfBase['H']))

dfBase = dfBase.drop('SO', 1)
dfBase = dfBase.drop('BB', 1)
dfBase = dfBase.drop('HR', 1)
dfBase = dfBase.drop('H', 1)
dfBase = dfBase.drop('2B', 1)
dfBase = dfBase.drop('R', 1)
dfBase = dfBase.drop('RBI', 1)
dfBase = dfBase.drop('SB', 1)
dfBase = dfBase.drop('CS', 1)
dfBase = dfBase.drop('3B', 1)
dfBase = dfBase.drop('TB', 1)
dfBase = dfBase.drop('AB', 1)

dfBase = dfBase.rename(columns={'OPS+':'OPSplus'})

dfBase = dfBase.dropna()

dfBase.info()
dfBase.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 1 to 1622
Data columns (total 13 columns):
G                101 non-null int64
PA               101 non-null int64
BA               101 non-null float64
OBP              101 non-null float64
SLG              101 non-null float64
OPS              101 non-null float64
OPSplus          101 non-null float64
SOper            101 non-null float64
BBper            101 non-null float64
HRper            101 non-null float64
SBper            101 non-null float64
RBIperAB         101 non-null float64
RunsGenerated    101 non-null float64
dtypes: float64(11), int64(2)
memory usage: 11.0 KB


,G,PA,BA,OBP,SLG,OPS,OPSplus,SOper,BBper,HRper,SBper,RBIperAB,RunsGenerated
1,156,675,0.304,0.354,0.552,0.906,140.0,0.191626,0.056361,0.053140,1.000000,0.164251,24.30
36,153,662,0.346,0.410,0.547,0.957,164.0,0.142373,0.098305,0.040678,0.842105,0.137288,36.85
49,146,606,0.257,0.276,0.402,0.679,80.0,0.275980,0.022147,0.028961,0.937500,0.095400,-21.85
52,158,689,0.297,0.337,0.471,0.808,108.0,0.157076,0.059098,0.031104,0.714286,0.136858,6.35
62,153,548,0.277,0.324,0.407,0.731,89.0,0.197628,0.071146,0.029644,0.666667,0.104743,-6.80


In [20]:
def forward_select(df, resp_str):
    
    remaining = set(df.columns)
    remaining.remove(resp_str)
    selected = []
    score_crnt, score_new = 0.0, 0.0
    while remaining and score_crnt == score_new:
        score_array = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(resp_str,' + '.join(selected + [candidate]))
            score = smf.ols(formula, df).fit().rsquared_adj
            score_array.append((score, candidate))
        score_array.sort()
        score_new, best_option = score_array.pop()
        if score_crnt < score_new:
            remaining.remove(best_option)
            selected.append(best_option)
            score_crnt = score_new
    formula = "{} ~ {} + 1".format(resp_str,' + '.join(selected))
    model = smf.ols(formula, df).fit()
    return model

In [23]:
model = forward_select(dfBase, "RunsGenerated")
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          RunsGenerated   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     3944.
Date:                Sat, 23 Dec 2017   Prob (F-statistic):          4.40e-112
Time:                        21:55:03   Log-Likelihood:                -151.76
No. Observations:                 101   AIC:                             319.5
Df Residuals:                      93   BIC:                             340.4
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   -150.7707      2.087    -72.236      0.000      -154.916  -146.626
OPS          214.3767      8.532     25.126      0.000       197.434   231.320
BBper        168.0884     17.448      9.634      0.000       133.440   202.737
OBP         -181.6204     30.904     -5.877      0.000      -242.990  -120.251
BA            99.2378     28.179      3.522      0.001        43.281   155.195
PA             0.0108      0.003      4.152      0.000         0.006     0.016
HRper        -35.9612     25.548     -1.408      0.163       -86.694    14.772
SBper         -0.5566      0.472     -1.178      0.242        -1.495     0.381
==============================================================================
Omnibus:                        8.426   Durbin-Watson:                   2.201
Prob(Omnibus):                  0.015   Jarque-Bera (JB):               18.047
Skew:                           0.019   Prob(JB):                     0.000121
Kurtosis:                       5.070   Cond. No.                     2.42e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.42e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
dfBase = dfBase.drop('SOper', 1)
dfBase = dfBase.drop('OPSplus', 1)

In [29]:
model = forward_select(dfBase, "RunsGenerated")
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          RunsGenerated   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     5421.
Date:                Sat, 23 Dec 2017   Prob (F-statistic):          5.06e-115
Time:                        22:22:32   Log-Likelihood:                -153.75
No. Observations:                 101   AIC:                             319.5
Df Residuals:                      95   BIC:                             335.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   -150.8720      2.101    -71.797      0.000      -155.044  -146.700
OPS          202.8742      2.529     80.216      0.000       197.853   207.895
BBper        167.5150     17.520      9.562      0.000       132.734   202.296
OBP         -170.1619     29.517     -5.765      0.000      -228.760  -111.564
BA           113.2785     27.039      4.189      0.000        59.600   166.957
PA             0.0105      0.003      3.995      0.000         0.005     0.016
==============================================================================
Omnibus:                        9.829   Durbin-Watson:                   2.109
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               23.941
Skew:                           0.075   Prob(JB):                     6.33e-06
Kurtosis:                       5.380   Cond. No.                     2.41e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.41e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [35]:
dfBaseminusPA = dfBase.drop('PA', 1)
dfBaseminusPA = dfBaseminusPA.drop('G', 1)
dfBaseminusPA = dfBaseminusPA.drop('OBP', 1)
dfBaseminusOBP = dfBase.drop('OBP', 1)

In [36]:
model = forward_select(dfBaseminusPA, "RunsGenerated")
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          RunsGenerated   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     5829.
Date:                Sat, 23 Dec 2017   Prob (F-statistic):          1.27e-113
Time:                        22:39:05   Log-Likelihood:                -161.85
No. Observations:                 101   AIC:                             333.7
Df Residuals:                      96   BIC:                             346.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   -145.3191      1.775    -81.881      0.000      -148.842  -141.796
OPS           38.6620     32.146      1.203      0.232       -25.148   102.472
BBper        166.7102     19.163      8.700      0.000       128.673   204.748
SLG          164.9648     32.412      5.090      0.000       100.628   229.302
BA           108.7432     29.537      3.682      0.000        50.112   167.375
==============================================================================
Omnibus:                       14.398   Durbin-Watson:                   1.931
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               42.596
Skew:                           0.310   Prob(JB):                     5.63e-10
Kurtosis:                       6.121   Cond. No.                         655.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
model = forward_select(dfBaseminusOBP, "RunsGenerated")
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          RunsGenerated   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     5374.
Date:                Sat, 23 Dec 2017   Prob (F-statistic):          7.69e-115
Time:                        22:36:37   Log-Likelihood:                -154.19
No. Observations:                 101   AIC:                             320.4
Df Residuals:                      95   BIC:                             336.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   -150.6367      2.134    -70.596      0.000      -154.873  -146.401
OPS           31.6107     30.009      1.053      0.295       -27.964    91.186
BBper        167.7941     17.859      9.395      0.000       132.340   203.249
SLG          171.3914     30.247      5.666      0.000       111.344   231.439
BA           113.6704     27.553      4.126      0.000        58.971   168.370
PA             0.0104      0.003      3.944      0.000         0.005     0.016
==============================================================================
Omnibus:                        9.518   Durbin-Watson:                   2.059
Prob(Omnibus):                  0.009   Jarque-Bera (JB):               21.927
Skew:                           0.115   Prob(JB):                     1.73e-05
Kurtosis:                       5.271   Cond. No.                     2.96e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.96e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""